In [1]:
import pandas as pd
import numpy as np
import os
from plotnine import *
import matplotlib.pyplot as plt

In [99]:
import warnings 
warnings.filterwarnings('ignore')

In [51]:
def alt_sign(x):
    "Alternate +1/-1 if x is even/odd"
    return (-1) ** x
def vio_box_line_plot(data,gene,lsize = 0.5,fill_alpha = 0.6, shift = 0.1):
    m1 = aes(x=stage('Type', after_scale='x+shift*alt_sign(x)'), fill='Type')              # shift outward
    m2 = aes(x=stage('Type', after_scale='x-shift*alt_sign(x)'), group='Sample ID', color = 'Sample')   # shift inward
    # plotnine.scales.scale_color_cmap(cmap_name='tab20', lut=None)
    
    fig = (ggplot(data, aes('Type', gene))
     + geom_violin(m1, style='left-right', draw_quantiles = [0.25,0.5,0.75], 
                   alpha=fill_alpha, size=lsize, color='#57606f')
     + geom_point(m2, alpha=fill_alpha, size=lsize)
     + geom_line(m2, size=lsize, alpha=fill_alpha)
     + geom_boxplot(aes(fill='Type'), width=shift, alpha=fill_alpha, size=lsize, show_legend=False, color='#57606f')
     + scale_fill_manual(["#FF5051","#CAE8F2"])
     + theme_classic()
     # + scale_color_manual(values = c("red","cyan","green","magenta","blue","yellow","orange","teal","purple","brown"))
#      + scale_color_manual(["#287271","#8AB07D","#E8A261","#FF5051","#3951A2",
# "#CAE8F2","#91D542","#F6E620","#780096","#CC4996"])
    + scale_color_manual(["#bcbd22","#7f7f7f","#9467bd","#1f77b4","#8c564b",
"#17becf","#e377c2","#d62728","#2ca02c","#ff7f0e"])
     # + scale_color_hue(h.start = 90)
     # + scale_color_cmap(cmap_name='RdYlGn')
     + theme(figure_size=(8, 6))
    )
    return fig

In [95]:
def test(data,gene,lsize = 0.5,fill_alpha = 0.6, shift = 0.1, psize=0.6):
    m1 = aes(x=stage('Type', after_scale='x+shift*alt_sign(x)'), fill='Type')              # shift outward
    m2 = aes(x=stage('Type', after_scale='x-shift*alt_sign(x)'), group='Sample ID', color = 'Sample')   # shift inward
    # plotnine.scales.scale_color_cmap(cmap_name='tab20', lut=None)
    #data, aes('Type', gene)
    fig = (ggplot()
     + geom_violin(data, aes('Type',gene,fill='Type'),style='left-right', alpha=fill_alpha,draw_quantiles = [0.25,0.5,0.75], size=lsize, color='#57606f')
     + geom_point(data, aes('Type',gene,group='Sample ID',color='Sample'), alpha=fill_alpha, size=psize)
     + geom_line(data, aes('Type',gene,group='Sample ID',color='Sample'), size=lsize, alpha=fill_alpha)
     # + geom_boxplot(aes(fill='Type'), width=shift, alpha=fill_alpha, size=lsize, show_legend=False, color='#57606f')
     + scale_fill_manual(["#FF5051","#CAE8F2"])
     + theme_classic()
     # + scale_color_manual(values = c("red","cyan","green","magenta","blue","yellow","orange","teal","purple","brown"))
     + scale_color_manual(["#bebf28","#7f7f7f","#9467bd","#2372A9","#8c564b",
"#4fcdd7","#e377c2","#d62728","#2ca02c","#ff7f0e"])
     # + scale_color_hue(h.start = 90)
     # + scale_color_cmap(cmap_name='RdYlGn')
     + theme(figure_size=(8, 6))
    )
    return fig

In [54]:
group = pd.read_csv('/data1/DYY/bambu/288_group.csv',sep='\t')
fdr_data = pd.read_csv('/data1/DYY/bambu/288_FDR/all_gene.csv', sep='\t').drop(['p_adjust','P_value'], axis=1).T
fdr_df = fdr_data.reset_index().rename(columns={'index':'name'}).merge(group).set_index('name').sort_values('Sample ID')

In [66]:
std = fdr_df.loc[fdr_df.Type=='Tumor'].drop(['Type','Sample ID'],axis=1).std()-fdr_df.loc[fdr_df.Type=='Normal'].drop(['Type','Sample ID'],axis=1).std()

In [74]:
fdr_df = fdr_df.T.drop(['Type','Sample ID'])

In [78]:
fdr_df.loc[:,'std'] = std

In [83]:
df = fdr_df.copy()

In [112]:
df = df.astype(float)

In [114]:
data = df.T.reset_index().rename(columns={'index':'name'})


data = pd.merge(data, group, on='name')
data.loc[:,'Sample'] = data['Sample ID'].str[:3]
# data1 = pd.merge(data1, group, on='name')

In [324]:

# plt.title('C19orf48')
p = vio_box_line_plot(data,'ENSG00000167747.17')
plt.savefig('/data1/DYY/bambu/plot/figure2/2d_down.pdf',dpi=400)

In [328]:
p.save('/data1/DYY/bambu/plot/figure2/2d_down.pdf',dpi=400)

In [322]:
vio_box_line_plot(data,'ENSG00000167747.17')
plt.savefig('/data1/DYY/bambu/plot/figure2/2d_down.pdf',dpi=400)

In [244]:
deseq = pd.read_csv('/data1/DYY/bambu/288_count_matrix_deseq2/all_result.csv',sep='\t')

In [128]:
deseq = deseq.loc[deseq.padj<0.05]

In [130]:
deseq

,row,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
2,BambuGene100001,8.283040,0.607993,0.121933,4.986303,6.154548e-07,0.000009
4,BambuGene100010,7.614386,1.071566,0.201297,5.323301,1.019009e-07,0.000002
9,BambuGene100063,8.204087,0.551757,0.116818,4.723226,2.321331e-06,0.000030
13,BambuGene100083,6.574575,0.594125,0.161961,3.668325,2.441450e-04,0.001781
19,BambuGene100154,12.975198,-0.271564,0.086362,-3.144478,1.663833e-03,0.009253
...,...,...,...,...,...,...,...
76805,ENSG00000291152.1,50.940436,-0.732595,0.221466,-3.307926,9.398950e-04,0.005711
76823,ENSG00000291173.1,16.778196,-0.540386,0.129577,-4.170386,3.040842e-05,0.000291
76830,ENSG00000291180.1,1.205464,-1.086423,0.332985,-3.262674,1.103663e-03,0.006544
76884,ENSG00000291234.1,9.856599,0.843879,0.315221,2.677106,7.426119e-03,0.031734


In [304]:
sca.loc[sca[0]=='ENSG00000167747.17']

,0
22,ENSG00000167747.17


In [138]:
deseq_non = deseq.loc[(deseq.log2FoldChange>-1) & (deseq.log2FoldChange<1)]

In [294]:
deseq_non.loc[deseq_non['row']=='ENSG00000167747.17']

,row,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
26428,ENSG00000167747.17,113.194071,0.968807,0.091358,10.604562,2.837898e-26,1.746544e-23


In [16]:
desgene = pd.read_csv('/data1/DYY/bambu/288_count_matrix_deseq2/all_norm.counts.fn',sep='\t')#.drop('gene_name',axis=1).set_index(['ensg'])

In [22]:
gene_name = pd.read_csv('/data1/DYY/data/ensg_gene_name.csv',sep='\t' ).drop_duplicates()

In [296]:
gene_name.loc[gene_name['1']=='ENSG00000167747.17']

,1,7
226216,ENSG00000167747.17,C19orf48


In [31]:
conv = pd.read_csv('/data1/DYY/bambu/288_conver_scatt_gene_all_tis/all_con.txt',sep='\t',header=None)

In [39]:
sca = pd.read_csv('/data1/DYY/bambu/288_conver_scatt_gene_all_tis/all_sca.txt',sep='\t',header=None)

In [35]:
conv.loc[conv[0]=='']

,0
930,ENSG00000163586.10


In [41]:
sca.loc[sca[0]=='ENSG00000111640.15']

,0


In [282]:
sca_df = pd.DataFrame(index=deseq.row)
for t in tissue:
    try:
        sca_ = pd.read_csv(f'/data1/DYY/bambu/288_conver_scatt_gene_all_tis/{t}_sca.txt',header=None)#.rename(columns={0:file[:3]})
    except:
        continue
    try:
        sca_df.loc[sca_[0],t] = 1
    except:
        # sca_df = sca_.copy()
        continue

In [284]:
sca_df = sca_df.loc[sca_df.count(axis=1)>0]

In [286]:
sca_df.loc[sca_df.count(axis=1)>2]

,BLA,BRE,COL,END,LIV,LUN,REC,KID
row,,,,,,,,
ENSG00000099953.10,NaN,1.0,1.0,NaN,1.0,NaN,NaN,NaN
ENSG00000164283.13,NaN,1.0,NaN,1.0,NaN,NaN,NaN,1.0
ENSG00000167747.17,NaN,NaN,1.0,NaN,NaN,NaN,1.0,1.0


In [264]:
sca_df.loc[(sca_df['all']==1) & (sca_df.count(axis=1)>3)]

,LIV,all,COL,REC,CER,LUN,BRE,END,KID,BLA
row,,,,,,,,,,
ENSG00000107562.17,1.0,1.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,1.0
ENSG00000164283.13,NaN,1.0,NaN,NaN,NaN,NaN,1.0,1.0,1.0,NaN
ENSG00000165197.5,NaN,1.0,1.0,NaN,NaN,NaN,1.0,NaN,NaN,1.0
ENSG00000167747.17,NaN,1.0,1.0,1.0,NaN,NaN,NaN,NaN,1.0,NaN
ENSG00000189129.14,NaN,1.0,1.0,NaN,NaN,NaN,1.0,NaN,1.0,1.0
ENSG00000198417.7,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN
ENSG00000205358.4,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN


In [254]:
sca_df.loc['ENSG00000126353.3']

LIV    NaN
all    1.0
COL    NaN
REC    NaN
CER    NaN
LUN    NaN
BRE    NaN
END    NaN
KID    NaN
BLA    NaN
Name: ENSG00000126353.3, dtype: float64

# GO conver

In [165]:
tissue = group['Sample ID'].str[:3].drop_duplicates()

In [183]:
go_df = pd.DataFrame()
for t in tissue:
    try:
        go = pd.read_csv(f'/home/bwu4/bambu_data_kegg_go/{t}_conv_go_bp.csv',sep='\t')
        go.loc[:,'tissue'] = t
        
        go_df = pd.concat([go_df,go])
    except:
        continue
    # break

In [187]:
go_df.to_csv('/data1/DYY/bambu/288_con_sca_enrichment_plot_data/merge_go.csv',sep='\t',index=False)

In [192]:
past_group = pd.read_csv('/data1/DYY/data/288_group.csv',sep='\t')
past_group['tissue'] = past_group['Sample ID'].str[:3]

In [198]:
group['tissue'] = group['Sample ID'].str[:3]

In [206]:
pd.concat([past_group.groupby('tissue').count()[['name']].rename(columns={'name':'past'}),group.groupby('tissue').count()[['name']]],axis=1)

,past,name
tissue,,
BLA,38,30
BRE,34,32
CER,20,32
COL,36,32
END,14,12
KID,36,40
LIV,38,34
LUN,34,38
REC,14,12


,name
tissue,
BLA,30
BRE,32
CER,32
COL,32
END,12
KID,40
LIV,34
LUN,38
REC,12
